In [7]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing
import csv

In [17]:
with open(r'M:\Documents\Courses\Credits\2019August\Computational Data Analysis\Module5\Exercises 5\Data\faces.csv', newline='') as csvfile:
    data = np.array(list(csv.reader(csvfile))).astype(float)

# Question1

In [39]:
# 1(a)

# center the data
data_c = data - np.mean(data, axis=0)

n, p = np.shape(data)

# Compute PCA as an eigenvalue analysis of the covariance matrix
Eval, Evec = np.linalg.eig(np.cov(data.T))

# sort the eigenvalues 
index = np.argsort(Eval)[::-1]

Eval = Eval[index].astype(np.float64)
Evec = Evec[:,index].astype(np.float64)

print(np.shape(Eval))

# Discard strictly none-positive eigenvalues modes
Eval = Eval[np.where(Eval > 1e-9)]
Evec = Evec[:, 0: len(Eval)]
print(np.shape(Eval))


(116,)
(36,)


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]


In [42]:
u, d, v = np.linalg.svd(data_c)

# calculate the variances
# keep only modes correspoding to strictly positive singular values -- diagonal matrix of the singular values are the eigen
# values
print(np.shape(d))
d = d[np.where(d > 1e-9)]
print(np.shape(d))

k = len(d)
print(np.shape(u), np.shape(v))
u = u[:, :k]
v = v[:k, :].T # Matrix that is returned from svd is ordered differently, therefor different slicing
print(np.shape(u), np.shape(v))

     # Assign PCA
L = v # the loading
S = np.matmul(u, np.diag(d)) # The scores
sigma2 = d ** 2 / n

# Check if values corrosponds
print("Differences in eigenvalues (variance): %2.3f" % (np.linalg.norm((Eval - d ** 2 / n), ord = 2)))
print("Difference in eigenvalues: %2.3f" % (np.linalg.norm(Evec ** 2 - v ** 2)))

(37,)
(36,)
(37, 37) (116, 116)
(37, 36) (116, 36)


In [65]:
print(d)

[0.31957907 0.16953028 0.13038426 0.12198532 0.09997679 0.09536748
 0.08765642 0.07998127 0.07380248 0.06899452 0.06324186 0.05993379
 0.05657064 0.05407138 0.05077074 0.04614636 0.04378579 0.04294402
 0.03802202 0.03732799 0.03333875 0.0313944  0.03023779 0.02876824
 0.02830552 0.02764182 0.02539063 0.02332958 0.02217975 0.02072585
 0.0200828  0.01878136 0.0170452  0.0162239  0.01583516 0.01394442]


# Question 2

In [60]:
# 2(a) - 

# Thresholding

# threshold the loadings - where the value is less than 0.15, it is replaced with 0
Lthres = L
Lthres[np.where(np.abs(L) < 0.15)] = 0

# find the new score matrix
Sthres = np.matmul(data_c, Lthres) # this is only an approximation, not exactly the correct way to determine the 
# score matrix since the loadings are not independent anymore. 
sigma2thres  = np.var(Sthres, axis = 0) # variance of the new score matrix





(116, 36) (116, 36)
(37, 36)


In [61]:
# 2(b)

# varimax

def varimax(Phi, gamma=1, q=20, tol=1e-6):
    from numpy import eye, asarray, dot, sum, diag
    from numpy.linalg import svd
    p,k = Phi.shape
    R = eye(k)
    d=0
    for i in range(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, diag(diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)  # rotation matrix
        d = sum(s)
        if d/d_old < tol: break
    return dot(Phi, R)

In [63]:
L_varimax = varimax(L[:, :12]) # k - number of columns - defines the sparsity
S_varimax = np.matmul(data_c, L_varimax) 
sigma2_varimax = np.var(S_varimax, axis = 0)



C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars


In [67]:
# 2(c)

""" Elastic net to choose components """
k = 12 # estimate first 12 components
L_en = np.zeros((p,k))
ElasticNet = linear_model.ElasticNet(alpha = 0.0001, l1_ratio = 0.1, fit_intercept = False) # Ratio is 0 for only l2 penalty
for i in range(k):
    reg_elastic = ElasticNet.fit(data_c, S[:,i]).coef_.reshape(-1,1)
    L_en[:,i] = preprocessing.normalize(reg_elastic, norm = "l2").ravel()
    
S_en = np.matmul(data_c, L_en)
sigma2_en = np.var(S_en, axis = 0)